# Exploring English Lexicon-based Tokenizer with no spaces

- TODO


In [1]:
import os, sys
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

#from importlib import reload  # Python 3.4+

import pickle
import pandas as pd

#force reimport
if 'pygents.util' in sys.modules:
    del sys.modules['pygents.util']
if 'pygents.text' in sys.modules:
    del sys.modules['pygents.text']
if 'pygents.plot' in sys.modules:
    del sys.modules['pygents.plot']
if 'pygents.token' in sys.modules:
    del sys.modules['pygents.token']
if 'pygents.token_plot' in sys.modules:
    del sys.modules['pygents.token_plot']


from pygents.token import *
from pygents.text import *
from pygents.util import *
from pygents.plot import plot_bars, plot_dict, matrix_plot
from pygents.token_plot import *

In [2]:
path = '../../nlp/corpora/Chinese/'
test_df = pd.read_csv(os.path.join(path,'magicdata/zh_en_ru_100/CORPUS_ZH_EN_RU.txt'),delimiter='\t')
test_texts = list(test_df['en'])
print(len(test_texts))
test_df[['en']]

100


,en
0,What about medical insurance? As for my family...
1,"For those who have insurance, according to the..."
2,Need to realize the importance of having insur...
3,"In fact, this phenomenon is indeed very common..."
4,It is really necessary for this generation of ...
...,...
95,Ant Insurance does not only offer car insuranc...
96,"However, when buying a house, except for the d..."
97,This kind of financial investment has certain ...
98,"If your investment orientation is right, then ..."


In [3]:
for text in test_texts:
    print(text)

What about medical insurance? As for my family, either an adult or a child will buy insurance.
For those who have insurance, according to the insurance contract, they will get a compensation of 300 thousand yuan.
Need to realize the importance of having insurance.
In fact, this phenomenon is indeed very common, for instance, for personal accident insurance, the more you buy, the more you insure.
It is really necessary for this generation of parents to buy insurance.
Well, right now, it's really advisable to buy insurance.
A car must be bought in full, and a house can be bought with a loan.
You can buy insurance, insurance is of course divided into many categories.
Medical insurance is very important.
It's the insurance company that pays this part of the money.
Xianghubao, I don't know if you ever heard about it, it is insurance in Alipay.
Buying a house is actually an investment.
Have you ever learned about the training of Ping An Insurance?
If it is deposited in the bank, what is the 

In [4]:
del_tokenizer = DelimiterTokenizer()


In [5]:
#get raw lexicon list
en_lex = list(pd.read_csv("https://raw.githubusercontent.com/aigents/aigents-java/master/lexicon_english.txt",sep='\t',header=None,na_filter=False).to_records(index=False))
print(len(en_lex))

#debug raw lexicon
print(max(en_lex,key=lambda item:item[1]))
en_lex_dict = weightedlist2dict(en_lex,lower=False) # no case-insensitive merge
print(len(en_lex_dict))

# merge and get top weight
en_lex_dict = weightedlist2dict(en_lex,lower=True) # with case-insensitive merge
top_weight = max([en_lex_dict[key] for key in en_lex_dict],key=lambda item:item)
print(top_weight)

# add delimiters to the list
en_lex_delimited = en_lex + [(i, top_weight) for i in list(delimiters)]
print(len(delimiters))
print(len(en_lex_delimited)) 


97565
('the', 53097401)
97565
53097401
34
97599


In [6]:
# no delimiters
filter_thresholds = [0,0.00001,0.0001,0.001,0.01]
for t in filter_thresholds:
    lex = listofpairs_compress_with_loss(en_lex,t) if t > 0 else en_lex
    en_lex0_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=0,cased=True)
    en_lex1_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=1,cased=True)
    en_lex2_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=2,cased=True)
    print(t,en_lex0_tokenizer.count_params())
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,en_lex0_tokenizer,nospaces=True,debug=False))#sort by len
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,en_lex1_tokenizer,nospaces=True,debug=False))#sort by freq
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,en_lex2_tokenizer,nospaces=True,debug=False))#sort by len and freq
    print()


0 97565
0.54
0.32
0.79

1e-05 40382
0.62
0.32
0.79

0.0001 10122
0.76
0.32
0.8

0.001 1570
0.57
0.32
0.57

0.01 118
0.31
0.26
0.31



In [7]:
# with delimiters
filter_thresholds = [0,0.00001,0.0001,0.001,0.01]
for t in filter_thresholds:
    lex = listofpairs_compress_with_loss(en_lex_delimited,t) if t > 0 else en_lex_delimited
    en_lex0_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=0,cased=True)
    en_lex1_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=1,cased=True)
    en_lex2_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=2,cased=True)
    print(t,en_lex0_tokenizer.count_params())
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,en_lex0_tokenizer,nospaces=True,debug=False))#sort by len
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,en_lex1_tokenizer,nospaces=True,debug=False))#sort by freq
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,en_lex2_tokenizer,nospaces=True,debug=False))#sort by len and freq
    print()


0 97599
0.54
0.32
0.79

1e-05 40416
0.62
0.32
0.79

0.0001 10156
0.76
0.32
0.8

0.001 1604
0.57
0.32
0.57

0.01 152
0.37
0.32
0.37



In [8]:
en_lex0_tokenizer = LexiconIndexedTokenizer(lexicon=en_lex_delimited,sortmode=0,cased=True)
for text in test_texts:
    expected = del_tokenizer.tokenize(text)
    remove_all(expected,' ')
    actual = en_lex0_tokenizer.tokenize(text.replace(' ',''))
    f1 = calc_f1(expected,actual)
    if f1 < 1:
        print(expected)
        print(actual)
        print(round(f1,2))


['What', 'about', 'medical', 'insurance', '?', 'As', 'for', 'my', 'family', ',', 'either', 'an', 'adult', 'or', 'a', 'child', 'will', 'buy', 'insurance', '.']
['What', 'about', 'medical', 'insurance', '?', 'Asf', 'orm', 'yf', 'ami', 'ly', ',', 'either', 'ana', 'du', 'lt', 'ora', 'child', 'will', 'buy', 'insurance', '.']
0.59
['For', 'those', 'who', 'have', 'insurance', ',', 'according', 'to', 'the', 'insurance', 'contract', ',', 'they', 'will', 'get', 'a', 'compensation', 'of', '300', 'thousand', 'yuan', '.']
['Forth', 'ose', 'who', 'have', 'insurance', ',', 'according', 'tothe', 'insurance', 'contract', ',', 'they', 'will', 'geta', 'compensation', 'of', '300', 'thousand', 'yuan', '.']
0.76
['Need', 'to', 'realize', 'the', 'importance', 'of', 'having', 'insurance', '.']
['Need', 'tore', 'ali', 'ze', 'thei', 'mpo', 'rta', 'nce', 'of', 'having', 'insurance', '.']
0.48
['In', 'fact', ',', 'this', 'phenomenon', 'is', 'indeed', 'very', 'common', ',', 'for', 'instance', ',', 'for', 'personal

['As', 'for', 'a', 'bond', 'fund', ',', 'it', 'offers', 'a', 'relatively', 'stable', 'profit', ',', 'which', 'is', 'just', 'like', 'a', 'fixed', 'deposit', 'but', 'with', 'more', 'benefits', '.']
['Asf', 'ora', 'bond', 'fund', ',', 'ito', 'ff', 'ers', 'are', 'lat', 'ive', 'lys', 'table', 'profit', ',', 'which', 'is', 'just', 'like', 'afi', 'xe', 'dde', 'posit', 'but', 'with', 'more', 'benefits', '.']
0.53
['How', 'many', 'such', 'acceptance', 'drafts', 'can', 'theoretically', 'be', 'issued', '.']
['How', 'many', 'such', 'acceptance', 'drafts', 'cant', 'he', 'ore', 'tical', 'ly', 'bei', 'ssu', 'ed', '.']
0.5
['It', 'means', 'that', 'each', 'of', 'us', 'should', 'have', 'our', 'way', 'to', 'manage', 'money', ',', 'no', 'matter', 'in', 'which', 'way', ',', 'no', 'matter', 'how', 'much', 'money', 'we', 'have', '.']
['Itm', 'ean', 'sth', 'ate', 'ach', 'of', 'uss', 'hould', 'have', 'our', 'way', 'toman', 'age', 'money', ',', 'noma', 'tte', 'rin', 'which', 'way', ',', 'noma', 'tte', 'rho', 'w

['The', 'insured', 'pays', 'premiums', 'to', 'obtain', 'compensation', 'within', 'the', 'scope', 'of', 'an', 'insurance', 'contract', '.']
['Thein', 'sure', 'dpa', 'ys', 'premiums', 'too', 'bta', 'inco', 'mpe', 'nsa', 'tion', 'within', 'thes', 'cope', 'ofa', 'nin', 'sura', 'nce', 'contract', '.']
0.23
['The', 'insurance', 'company', 'will', 'not', 'pay', 'you', 'compensation', 'in', 'any', 'amount', '.']
['Thein', 'sura', 'nce', 'company', 'will', 'not', 'pay', 'you', 'compensation', 'ina', 'nya', 'mount', '.']
0.56
['Social', 'insurance', 'means', 'the', 'five', 'social', 'insurance', 'and', 'the', 'housing', 'provident', 'fund', 'which', 'are', 'mandatory', '.']
['Social', 'insurance', 'means', 'the', 'fives', 'oc', 'ial', 'insurance', 'andthe', 'housing', 'provident', 'fund', 'which', 'are', 'mandatory', '.']
0.75
['through', 'the', 'change', 'of', 'the', 'benchmark', 'interest', 'rate']
['through', 'the', 'change', 'ofthe', 'benchmark', 'interest', 'rate']
0.8
['Well', ',', 'issue'